<a href="https://colab.research.google.com/github/jcms2665/GoogleEarthEngine/blob/master/DL_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
  <img src="https://drive.google.com/uc?export=view&id=1578Q-90caRQKHlFMq8HEj2GIGPLkX8-b" alt="centered image" />
</center>

<H1 align="center"> <b>Estimación de áreas pequeñas </b></H1>

Esta sección forma parte del análisis empírico del proyecto. La intención es consolidar un único documento en donde se concentre a la información geográfica y estadística.  Este es el primer ejercicio con **redes neuronales** para clasificar las imagenes de Google Earth con las siguientes características:


| Características del modelo   |            |
|----------|:-------------:|
| Modelo |  Secuencial |
| Épocas |    10   |
| Optimizador | Adam |
| Función de costo |    categorical_crossentropy   |
| Métrica | accuracy |



Se utilizaron los tutoriales de GEE y *tensorflow*.


In [ ]:
# Librerías
import ee
import tensorflow as tf
import folium
import time

# Token
from google.colab import auth
auth.authenticate_user()
ee.Authenticate()
ee.Initialize()

# Definición de objetos

In [ ]:
# Acá se guardan todos los objetos que se utilizan en el proceso

# 0. Usuario de GCloud

OUTPUT_BUCKET = 'jcms2665_2'

# 1. Imágenes 

L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

#1.1 Características 
      # Base etiquetada de prueba y validación
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')

      # Propiedad (variable) en donde se guarda la etiqueta
LABEL = 'landcover'; N_CLASSES = 3

      # Lista de las Bandas y sus etiquetas
FEATURE_NAMES = list(BANDS); FEATURE_NAMES.append(LABEL)

#1.2 Archivos creados

      # Archivos TFRecord que se exportarán a GEE y GCloud
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

      # Archivo de predicción (imagen). 
      # El modelo entrenado leerá archivo y clasificará los píxeles.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

      # Archivo con las imágenes clasificadas
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'

      # Exportar imágenes en esta región: Corredor
EXPORT_REGION = ee.Geometry.Rectangle([-98.01, 20.4, -102.35, 21.27])

      # Archivo que se creará al exportar los datos a este archivo
OUTPUT_ASSET_ID = 'users/' + 'jcms2665' + '/Classified_pixel_demo'

# Datos de GEE

In [ ]:
# Función máscara nubes
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

# Rango de fechas y mediana de las imágenes
image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Mostrar mapa y vincular con GEE
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[21.16, -100.79])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# Muestras aleatorias
sample = image.sampleRegions(
  collection=LABEL_DATA, properties=[LABEL], scale=10).randomColumn()

# Partición de la muestra 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.3))

from pprint import pprint

# Verificación: se imprime la primera pareja de puntos.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

In [ ]:
# Acceso a mi cuenta de google Cloud
print('Acceso.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
    else 'Sin acceso')

In [ ]:
# Crear las tareas (funciones)
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

# Exportación
training_task.start()
testing_task.start()

In [ ]:
# Imprimir
pprint(ee.batch.Task.list())

In [ ]:
print('Listo datos de entrenamiento.' if tf.io.gfile.exists(TRAIN_FILE_PATH) 
    else 'No')
print('Listo datos de prueba.' if tf.io.gfile.exists(TEST_FILE_PATH) 
    else 'No')

## Imágenes



In [ ]:
# Dimensiones y el archivo

image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=30,
  fileFormat='TFRecord',
  region=EXPORT_REGION.toGeoJSON()['coordinates'],
  formatOptions=image_export_options,
)

# Ejecución de la tarea
image_task.start()

In [ ]:
# Imprimir
pprint(ee.batch.Task.list())

In [ ]:
while image_task.active():
  print('sondeo para la tarea (id: {}).'.format(image_task.id))
  time.sleep(30)
print('Exportación de las imágenes listas')

## Preparación y preprocesamiento de datos



In [ ]:
# Cree un conjunto de datos a partir del archivo TFRecord en Cloud Storagee.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# Imprima el primer registro para verificar.
print(iter(train_dataset).next())

## Estructura de sus datos



In [ ]:
# Lista de características de longitud fija, todas las cuales son float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Diccionario con nombres como claves, características como valores.
features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

## Analizar el conjunto de datos



In [ ]:
def parse_tfrecord(example_proto):
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Función sobre el conjunto de datos.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Imprima el primer registro analizado para comprobarlo.
pprint(iter(parsed_dataset).next())

## NDVI


In [ ]:
def normalized_difference(a, b):
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
  features['NDVI'] = normalized_difference(features['B5'], features['B4'])
  return features, label

# Modelo



## Keras


In [ ]:
from tensorflow import keras

# Gregar el NDVI.
input_dataset = parsed_dataset.map(add_NDVI)

# Keras requiere entradas como una tupla. Función de costo
# denominada categorical_crossentropy, la etiqueta necesita ser cambiada a una
# forma vectorial

def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Asigne la función to_tuple, shuffle y batch.
input_dataset = input_dataset.map(to_tuple).batch(8)

# Definir las capas del modelo.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

# Compilar el modelo con la función de costo, el optimizador y la métrica.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 10 épocas
model.fit(x=input_dataset, epochs=10)

## Precisión

In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(add_NDVI)
    .map(to_tuple)
    .batch(1))

model.evaluate(test_dataset)

# Entrenar al modelo para clasificar una imagen de Earth Engine



In [ ]:
# Lista de todos los archivos que están en google cloud
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}

# Obtener solo los archivos generados por la importación de imágenes
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Obtenga la lista de archivos de imagen y el archivo del mezclador JSON.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Los archivos deben estar en el orden correcto
image_files_list.sort()

pprint(image_files_list)
print(json_file)

In [ ]:
import json

# Cargue el contenido del archivo del mezclador en un objeto JSON.
json_text = !gsutil cat {json_file}
# Obtenga una sola cadena con / newlines de IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Los tensores tienen la forma de un parche, un parche para cada banda.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

# Diccionario de análisis.
image_features_dict = dict(zip(BANDS, image_columns))

# Tenga en cuenta que puede crear un conjunto de datos a partir de muchos archivos especificando una lista.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Función de análisis.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Analice los datos en tensores, un tensor largo por parche.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Divide nuestros tensores largos en muchos pequeños.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Agregue funciones adicionales (NDVI).
image_dataset = image_dataset.map(
  # Agregue NDVI a una función que no tiene etiqueta.
  lambda features: add_NDVI(features, None)[0]
)

# Convierta el diccionario de cada registro en una tupla sin etiqueta.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Convierta cada parche en un lote.
image_dataset = image_dataset.batch(patch_width * patch_height)

## Predicciones para los píxeles

In [ ]:
# Predicción por lotes, con tantos pasos como parches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

print(predictions[0])

## Predicciones 

In [ ]:
print('Writing to file ' + OUTPUT_IMAGE_FILE)

In [ ]:
# Inicializar writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Cada parche de predicciones, volcaremos un ejemplo en la salida.
# archivo con una sola característica que contiene nuestras predicciones. Dado que nuestras predicciones
# ya están en el orden de los datos exportados, los parches que creamos aquí
# también estará en el orden correcto.
patch = [[], [], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  # Una vez que hemos visto un valor de parches de class_ids ...
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # Crear ejemplos
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
        }
      )
    )

    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    cur_patch += 1

writer.close()

# Clasificación con datos de Earth Engine

In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

In [ ]:
print('Uploading to ' + OUTPUT_ASSET_ID)

In [ ]:
# Empezar la subida
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

In [ ]:
ee.batch.Task.list()

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']
}
probability_vis = {'bands': ['bareProb', 'vegProb', 'waterProb'], 'max': 0.5}

prediction_map_id = predictions_image.getMapId(prediction_vis)
probability_map_id = predictions_image.getMapId(probability_vis)

map = folium.Map(location=[21.16, -100.79])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probability_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map